In [1]:
##验证集：模型推理
import pandas as pd
# pd.set_option('display.max_colwidth', None)  # 显示完整的列宽
# pd.set_option('display.width', None)         # 不限制显示宽度
# pd.set_option('display.max_columns', None)   # 显示所有列
pd.reset_option('all')

/tmp/ipykernel_34482/1337929260.py:6: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/tmp/ipykernel_34482/1337929260.py:6: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [2]:
%%capture #不显示安装过程
import sys, torch
import pandas as pd 
print(sys.executable)
print(torch.__version__, torch.version.cuda)
!pip install huggingface-hub
!pip install unsloth
# Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install --upgrade unsloth

[HAMI-core Msg(34482:140591513917248:libvgpu.c:836)]: Initializing.....
[HAMI-core Warn(34482:140591513917248:multiprocess_memory_limit.c:592)]: Kick dead proc 31243
[HAMI-core Warn(34482:140591513917248:multiprocess_memory_limit.c:592)]: Kick dead proc 31383


In [3]:
test_data = pd.read_parquet("data/finetuning_data_test.parquet")

In [4]:
test_data

,UserID,instruction,input,output,text
0,3,Filter exactly 10 unique item IDs that the use...,,"Recommended movie ID: [1270, 1259, 1079, 1265,...",The following is the description of the recomm...
1,8,Filter exactly 10 unique item IDs that the use...,,"Recommended movie ID: [4, 1027, 3155, 42, 3500...",The following is the description of the recomm...
2,13,Filter exactly 10 unique item IDs that the use...,,"Recommended movie ID: [3269, 552, 2135, 1587, ...",The following is the description of the recomm...
3,18,Filter exactly 10 unique item IDs that the use...,,"Recommended movie ID: [2718, 3723, 2145, 261, ...",The following is the description of the recomm...
4,23,Filter exactly 10 unique item IDs that the use...,,"Recommended movie ID: [2259, 2409, 2410, 327, ...",The following is the description of the recomm...
...,...,...,...,...,...
1203,6018,Filter exactly 10 unique item IDs that the use...,,"Recommended movie ID: [2021, 3033, 3269, 674, ...",The following is the description of the recomm...
1204,6023,Filter exactly 10 unique item IDs that the use...,,"Recommended movie ID: [493, 1620, 3247, 2764, ...",The following is the description of the recomm...
1205,6028,Filter exactly 10 unique item IDs that the use...,,"Recommended movie ID: [1573, 592, 2427, 6, 110...",The following is the description of the recomm...
1206,6033,Filter exactly 10 unique item IDs that the use...,,"Recommended movie ID: [349, 780, 1608, 380, 13...",The following is the description of the recomm...


In [5]:
import pandas as pd
from unsloth import FastLanguageModel
import torch
import re
from tqdm import tqdm 

# 加载模型与 tokenizer（你已正确加载）
max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="outputs/lora_model",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
FastLanguageModel.for_inference(model)

# test_data = pd.read_csv("your_test_data.csv")  # 示例
##加载数据
# test_data = pd.read_parquet("data/finetuning_data_test.parquet")
test_data = test_data
results = []

for idx, row in tqdm(test_data.iterrows(), total=len(test_data), desc="推理生成："):
    content = row['instruction']
    
    messages = [
        {"role": "user", "content": content}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    # 不使用 streamer，改为直接获取输出
    outputs = model.generate(
        input_ids,
        max_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.01,
        do_sample=True,
    )

    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 提取最后的回答部分（去掉 prompt）
    answer = response_text.split("[/INST]")[-1].strip() if "[/INST]" in response_text else response_text.strip()
    
    # 使用正则表达式提取电影ID列表
    # 匹配 "Recommended movie ID: [数字, 数字, ...]" 格式
    pattern = r"Recommended movie ID:\s*\[([^\]]+)\]"
    match = re.search(pattern, answer)
    
    if match:
        # 提取方括号内的内容
        ids_str = match.group(1)
        # 分割并转换为整数列表
        try:
            item_ids = [int(id.strip()) for id in ids_str.split(',') if id.strip().isdigit()]
            # 保证最多10个唯一ID
            unique_item_ids = list(dict.fromkeys(item_ids))[:10]
        except:
            # 如果解析失败，回退到原来的按行分割方法
            item_ids = [line.strip() for line in answer.split("\n") if line.strip().isdigit()]
            unique_item_ids = list(dict.fromkeys(item_ids))[:10]
    else:
        # 如果没有找到推荐格式，回退到原来的按行分割方法
        item_ids = [line.strip() for line in answer.split("\n") if line.strip().isdigit()]
        unique_item_ids = list(dict.fromkeys(item_ids))[:10]

    results.append({
        "UserID": row.get("UserID", idx),  # 如果有 uid 列则使用，否则用索引
        "recommended_items": unique_item_ids  # 保存为列表格式
    })

# 转换为 DataFrame 并保存
result_df = pd.DataFrame(results)
# 保存时将列表转换为字符串格式以便CSV兼容
result_df['recommended_items_str'] = result_df['recommended_items'].apply(lambda x: ','.join(map(str, x)))
result_df.to_csv("data/recommended_results.csv", index=False)

print(" 推荐结果已保存至 recommended_results.csv")

[HAMI-core Msg(34482:140591513917248:libvgpu.c:852)]: Initialized


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


[HAMI-core Msg(34599:140062620075840:libvgpu.c:836)]: Initializing.....
/usr/local/lib/python3.12/dist-packages/transformers/loss/loss_for_object_detection.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.2)
  from scipy.optimize import linear_sum_assignment
[HAMI-core Msg(34599:140062620075840:libvgpu.c:852)]: Initialized


Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.25 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


[HAMI-core Msg(34482:140591513917248:memory.c:512)]: orig free=11507597312 total=25393692672 limit=24964497408 usage=822347776
[HAMI-core Msg(34482:140591513917248:memory.c:512)]: orig free=11507597312 total=25393692672 limit=24964497408 usage=822347776
Unsloth 2025.8.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
推理生成：: 100%|██████████| 1208/1208 [31:05<00:00,  1.54s/it]

 推荐结果已保存至 recommended_results.csv


In [6]:
result_df = pd.read_csv("data/recommended_results.csv")

In [7]:
result_df.head()

,UserID,recommended_items,recommended_items_str
0,3,"[3407, 1253, 1259, 1265, 1270, 1266, 362, 33, ...","3407,1253,1259,1265,1270,1266,362,33,3690,369"
1,8,"[1370, 50, 474, 1027, 790, 1, 2390, 3257, 3258]","1370,50,474,1027,790,1,2390,3257,3258"
2,13,"[1910, 552, 1587, 2135, 1375, 191, 1085, 2, 10...","1910,552,1587,2135,1375,191,1085,2,1009,2054"
3,18,[858],858
4,23,"[2643, 3740]","2643,3740"


In [15]:
##准备real_interected_items
real_items = pd.read_csv("data/real_interected_items.csv")

In [16]:
real_items.head()

,UserID,CandidateItems,InteractedItems
0,3,2019:Action|Drama;549:Documentary;377:Action|R...,1270;1259;1079;1265;2355;1641;3552;104;3868;2081
1,8,50:Crime|Thriller;3257:Action|Drama|Romance|Th...,4;1027;3155;42;3500;3213;1274;1;741;3257
2,13,1822:Children's|Comedy;2054:Adventure|Children...,3269;552;2135;1587;2054;2;1009;1375;2641;2822
3,18,3779:Sci-Fi;1366:Drama;1092:Mystery|Thriller;2...,2718;3723;2145;261;3448;838;26;266;454;1683
4,23,3131:Comedy;582:Comedy;3854:Drama|Romance;1388...,2259;2409;2410;327;1388;2411;2642;2149;2119;2643


In [17]:
import pandas as pd
try:
    real_items['InteractedItems'] = real_items['InteractedItems'].str.split(';').apply(lambda x: [int(item) for item in x if item])
except ValueError as e:
    print(f"Error converting 'InteractedItems' to list of integers: {e}")
    print("检查空值")

In [18]:
real_items.head()

,UserID,CandidateItems,InteractedItems
0,3,2019:Action|Drama;549:Documentary;377:Action|R...,"[1270, 1259, 1079, 1265, 2355, 1641, 3552, 104..."
1,8,50:Crime|Thriller;3257:Action|Drama|Romance|Th...,"[4, 1027, 3155, 42, 3500, 3213, 1274, 1, 741, ..."
2,13,1822:Children's|Comedy;2054:Adventure|Children...,"[3269, 552, 2135, 1587, 2054, 2, 1009, 1375, 2..."
3,18,3779:Sci-Fi;1366:Drama;1092:Mystery|Thriller;2...,"[2718, 3723, 2145, 261, 3448, 838, 26, 266, 45..."
4,23,3131:Comedy;582:Comedy;3854:Drama|Romance;1388...,"[2259, 2409, 2410, 327, 1388, 2411, 2642, 2149..."


In [19]:
results = pd.merge(result_df,real_items,on = 'UserID',how = 'inner')
results = results[['UserID', 'recommended_items', 'InteractedItems']]

In [20]:
results.head()

,UserID,recommended_items,InteractedItems
0,3,"[3407, 1253, 1259, 1265, 1270, 1266, 362, 33, ...","[1270, 1259, 1079, 1265, 2355, 1641, 3552, 104..."
1,8,"[1370, 50, 474, 1027, 790, 1, 2390, 3257, 3258]","[4, 1027, 3155, 42, 3500, 3213, 1274, 1, 741, ..."
2,13,"[1910, 552, 1587, 2135, 1375, 191, 1085, 2, 10...","[3269, 552, 2135, 1587, 2054, 2, 1009, 1375, 2..."
3,18,[858],"[2718, 3723, 2145, 261, 3448, 838, 26, 266, 45..."
4,23,"[2643, 3740]","[2259, 2409, 2410, 327, 1388, 2411, 2642, 2149..."


In [21]:
print("Type of 'recommended_items'[0]:", type(results['recommended_items'].iloc[0]))
print("Type of 'InteractedItems'[0]:", type(results['InteractedItems'].iloc[0]))

Type of 'recommended_items'[0]: <class 'str'>
Type of 'InteractedItems'[0]: <class 'list'>


## 计算模型精确率

In [22]:
import pandas as pd
import ast

def convert_str_to_list(s):
    """安全地将字符串形式的列表转换为真正的列表"""
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except (ValueError, SyntaxError):
            print(f"Warning: Could not parse string: {s}")
            return []
    elif isinstance(s, list):
        return s
    else:
        print(f"Warning: Expected string or list, got {type(s)}. Returning empty list.")
        return []

results['recommended_items'] = results['recommended_items'].apply(convert_str_to_list)

# --- 现在再检查类型 ---
print("Type of 'recommended_items'[0] after conversion:", type(results['recommended_items'].iloc[0]))
print("Type of 'InteractedItems'[0] (should still be list):", type(results['InteractedItems'].iloc[0]))

# --- 现在可以安全地计算交集了 ---
def calculate_overlap_ratio(row):
    try:
        # 确保现在两者都是列表
        recommended_set = set(row['recommended_items'])
        interacted_set = set(row['InteractedItems'])
        
        # 计算交集大小
        intersection_size = len(recommended_set.intersection(interacted_set))
        
        # 根据公式计算重复度 (交集大小 / 10)
        overlap_ratio = intersection_size / 10.0 
        
        return overlap_ratio
    except Exception as e: # 捕获所有可能的错误
        print(f"Error calculating overlap for row {row.name}: {e}")
        return 0.0



Type of 'recommended_items'[0] after conversion: <class 'list'>
Type of 'InteractedItems'[0] (should still be list): <class 'list'>


In [23]:
# 应用函数并计算平均值
results['overlap_ratio'] = results.apply(calculate_overlap_ratio, axis=1)
average_overlap_ratio = results['overlap_ratio'].mean()

print(f"Average Overlap Ratio (Mean): {average_overlap_ratio:.4f}")

Average Overlap Ratio (Mean): 0.2305


In [24]:
results.head()

,UserID,recommended_items,InteractedItems,overlap_ratio
0,3,"[3407, 1253, 1259, 1265, 1270, 1266, 362, 33, ...","[1270, 1259, 1079, 1265, 2355, 1641, 3552, 104...",0.3
1,8,"[1370, 50, 474, 1027, 790, 1, 2390, 3257, 3258]","[4, 1027, 3155, 42, 3500, 3213, 1274, 1, 741, ...",0.3
2,13,"[1910, 552, 1587, 2135, 1375, 191, 1085, 2, 10...","[3269, 552, 2135, 1587, 2054, 2, 1009, 1375, 2...",0.7
3,18,[858],"[2718, 3723, 2145, 261, 3448, 838, 26, 266, 45...",0.0
4,23,"[2643, 3740]","[2259, 2409, 2410, 327, 1388, 2411, 2642, 2149...",0.1


In [25]:
pd.set_option('display.max_colwidth', None)  # 显示完整的列宽
pd.set_option('display.width', None)         # 不限制显示宽度
pd.set_option('display.max_columns', None)   # 显示所有列

In [26]:
# 如果有 None 值（由错误处理产生），mean() 会自动忽略它们 (skipna=True 是默认行为)
average_overlap_ratio = results['overlap_ratio'].mean()

# 输出结果
print(f"Average Overlap Ratio (Mean): {average_overlap_ratio:.4f}")

Average Overlap Ratio (Mean): 0.2305
